references
1. https://github.com/ultralytics/ultralytics/issues/1693
2. https://discuss.pytorch.org/t/what-is-the-purpose-of-is-leaf/87000/3
3. https://pytorch.org/docs/master/autograd.html?highlight=grad#torch.autograd.grad
4. https://blog.paperspace.com/pytorch-101-understanding-graphs-and-automatic-differentiation/


In [3]:
from ultralytics import YOLO
from ultralytics.yolo.v8.detect.train import DetectionTrainer
from ultralytics.yolo.data.dataloaders.v5loader import create_dataloader
from ultralytics.yolo.v8.detect.train import Loss
import cv2
# import torch
# import matplotlib.pyplot as plt

In [ ]:
# load detection trainer using the weights best.pt
# best.pt is the custom model trained on YOLO_data_je
trainer = DetectionTrainer('args.yaml')

Ultralytics YOLOv8.0.82 🚀 Python-3.9.12 torch-1.12.1 CPU
yolo/engine/trainer: task=detect, mode=predict, model=../model_results/train/weights/best.pt, data=../detection_data.yaml, epochs=1, patience=50, batch=1, imgsz=832, save=False, save_period=-1, cache=False, device=None, workers=4, project=None, name=None, exist_ok=False, pretrained=False, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=1, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, si

In [3]:
trainer.setup_model()


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.Conv                  [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.Conv                  [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.C2f                   [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.Conv                  [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.C2f                   [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.Conv                  [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.C2f                   [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.Conv                  [128

{'epoch': -1,
 'best_fitness': None,
 'model': DetectionModel(
   (model): Sequential(
     (0): Conv(
       (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
       (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
       (act): SiLU(inplace=True)
     )
     (1): Conv(
       (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
       (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
       (act): SiLU(inplace=True)
     )
     (2): C2f(
       (cv1): Conv(
         (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
         (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
         (act): SiLU(inplace=True)
       )
       (cv2): Conv(
         (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
         (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track

In [4]:
trainer.set_model_attributes()

In [5]:
loader = create_dataloader('../data/YOLO_short/images', imgsz=832,
                           batch_size=1,
                           stride=1)

Scanning ../data/YOLO_short/labels.cache... 6 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6/6 [00:00<?, ?it/s]


In [6]:
batch = next(iter(loader[0]))
batch['img'].shape

torch.Size([1, 3, 832, 832])

In [7]:
batch['img']

tensor([[[[114, 114, 114,  ..., 114, 114, 114],
          [114, 114, 114,  ..., 114, 114, 114],
          [114, 114, 114,  ..., 114, 114, 114],
          ...,
          [114, 114, 114,  ..., 114, 114, 114],
          [114, 114, 114,  ..., 114, 114, 114],
          [114, 114, 114,  ..., 114, 114, 114]],

         [[114, 114, 114,  ..., 114, 114, 114],
          [114, 114, 114,  ..., 114, 114, 114],
          [114, 114, 114,  ..., 114, 114, 114],
          ...,
          [114, 114, 114,  ..., 114, 114, 114],
          [114, 114, 114,  ..., 114, 114, 114],
          [114, 114, 114,  ..., 114, 114, 114]],

         [[114, 114, 114,  ..., 114, 114, 114],
          [114, 114, 114,  ..., 114, 114, 114],
          [114, 114, 114,  ..., 114, 114, 114],
          ...,
          [114, 114, 114,  ..., 114, 114, 114],
          [114, 114, 114,  ..., 114, 114, 114],
          [114, 114, 114,  ..., 114, 114, 114]]]], dtype=torch.uint8)

In [8]:
# check if the batch['img'] is a leaf node; if it is .backward() accumulates grad
batch['img']=batch['img'].float()
print(batch['img'].is_leaf)
print(batch['img'].requires_grad)
batch['img'].requires_grad=True # additional argument to get the gradient of the input

True
False


In [9]:
# batch variable set from the loader has cls (true label of the images)
# it reads the data from data=../detection_data.yaml, in args.yaml when calling the trainer

batch 

{'ori_shape': ((28, 112),),
 'ratio_pad': (((7.428571428571429, 7.428571428571429), (0.0, 312.0)),),
 'im_file': ('../data/YOLO_short/images/0.png',),
 'img': tensor([[[[114., 114., 114.,  ..., 114., 114., 114.],
           [114., 114., 114.,  ..., 114., 114., 114.],
           [114., 114., 114.,  ..., 114., 114., 114.],
           ...,
           [114., 114., 114.,  ..., 114., 114., 114.],
           [114., 114., 114.,  ..., 114., 114., 114.],
           [114., 114., 114.,  ..., 114., 114., 114.]],
 
          [[114., 114., 114.,  ..., 114., 114., 114.],
           [114., 114., 114.,  ..., 114., 114., 114.],
           [114., 114., 114.,  ..., 114., 114., 114.],
           ...,
           [114., 114., 114.,  ..., 114., 114., 114.],
           [114., 114., 114.,  ..., 114., 114., 114.],
           [114., 114., 114.,  ..., 114., 114., 114.]],
 
          [[114., 114., 114.,  ..., 114., 114., 114.],
           [114., 114., 114.,  ..., 114., 114., 114.],
           [114., 114., 114.,  ...

In [10]:
# this is where backpropagation happens
a = trainer.model(batch['img'])
loss_fn = Loss(trainer.model)
loss, _ = loss_fn(a, batch)
loss.backward()

In [11]:
(batch['img'].grad).shape

torch.Size([1, 3, 832, 832])

In [13]:
batch['img'].grad # checked that the gradient w.r.t. input is deterministic

tensor([[[[ 4.6456e-05, -1.2548e-04, -3.9521e-04,  ...,  1.1777e-03,  1.5254e-03,  8.9410e-04],
          [ 1.0720e-04, -5.9228e-05, -2.5412e-04,  ...,  8.4059e-04,  1.1143e-03,  9.8174e-04],
          [ 2.0367e-04,  2.3792e-04,  2.1482e-04,  ..., -3.0040e-03, -1.4534e-03, -1.0063e-03],
          ...,
          [-9.1604e-04, -1.0704e-03, -3.0529e-03,  ...,  1.3474e-04,  2.3631e-04,  3.1245e-04],
          [-1.8301e-03, -2.5013e-03, -2.9528e-03,  ..., -3.1744e-04, -8.6755e-05,  3.2632e-05],
          [-1.5474e-03, -2.4287e-03, -2.4097e-03,  ..., -2.6404e-04, -2.6979e-05,  2.4986e-05]],

         [[-8.1715e-05,  3.3616e-05,  8.4861e-05,  ..., -8.2129e-04, -9.8399e-04, -1.0775e-03],
          [-1.4777e-04,  5.3441e-07,  7.1622e-05,  ...,  6.1321e-06, -2.5289e-04, -4.2269e-04],
          [-1.7335e-04, -1.8212e-04, -6.2230e-05,  ...,  1.2622e-03,  5.4315e-04,  5.7904e-04],
          ...,
          [ 6.6152e-04,  1.3595e-03,  8.0170e-04,  ..., -1.2668e-04, -2.1190e-04, -2.7022e-05],
        

In [13]:
# #load the model
# model = YOLO("model_results/train7/weights/best.pt")
# res = model(data)
# # model.train(task='detect', data='../detection_data.yaml',
# #              epochs=1, batch=1, augment=False)